## Matrix multiplication

There are many math problems in STEM that can be reduced to matrix multiplication.  We are going to use matrix multiplication for examples in future courses, so it is **important** that we understand how it works. In this exercise we will solidify our understanding of multi-dimensional indexing by fixing **two lines of code** in a matrix multiplication algorithm.

A matrix is a two-dimensional array. If we have two matrices **A** (of size N0_C, N1_A) and **B** (of size N1_A, N1_C), then we can multiply them to make a matrix **C** (of size N0_C, N1_C). Every pixel at coordinates (i0,i1) in **C** is obtained by taking a **dot product** of row **i0** in **A** with column **i1** in B. That means the elements of row **i0** in A are multiplied by the elements of column **i1** in B and a sum is taken over the products.

<figure style="margin-left:auto; margin-right:auto; width:80%;">
    <img style="vertical-align:middle" src="../../images/matrix_multiplication.svg">
    <figcaption style= "text-align:lower; margin:1em; float:bottom; vertical-align:bottom;">Multiplying matrices A and B to get C.</figcaption>
</figure>

The code [mat_mult.cpp](mat_mult.cpp) takes care of most things, such as creating memory allocations **A_h** and **B_h**, **C_h**; and filling **A_h** and **B_h** with random numbers. Then the code uses nested loops to visit every pixel in **C** (i0,i1) and calls a function (which we call a kernel) called **mat_mult** to perform the dot product for the value in **C** at coordinates (i0, i1). We use **row-major** ordering for this example.

<figure style="margin-left:auto; margin-right:auto; width:100%;">
    <img style="vertical-align:middle" src="../../images/matrix_kernel_indexing.svg">
    <figcaption style= "text-align:lower; margin:1em; float:bottom; vertical-align:bottom;">Computation of the dot product. Shown are the starting offsets and strides for matrices A and B with row-major ordering.</figcaption>
</figure>

Open [mat_mult.cpp](mat_mult.cpp) and see if you can follow the structure of the application. Once every pixel is visited, the code checks matrix **C** against a known answer. The kernel isn't working properly though and it will help  solidify understanding of multi-dimensional indexing and matrix multiplication if you work through the solution and determine the offsets into **A_h** and **B_h**. 

### The problem

The kernel function from line 17 of [mat_mult.cpp](mat_mult.cpp) is shown here:

```C++
// Kernel to perform the matrix multiplication
void mat_mult(
        float *A, 
        float *B, 
        float *C, 
        size_t N0_C, 
        size_t N1_C, 
        size_t N1_A, 
        size_t i0, 
        size_t i1) {

    // A is of size (N0_C, N1_A) = (4,3)
    // B is of size (N1_A, N1_C) = (3,6)   
    // C is of size (N0_C, N1_C) = (4,6)

    // Make sure we stay within the bounds of matrix C
    if ((i0<N0_C) && (i1<N1_C)) {
        
        // Define a loop constant
        float temp=0.0;

        // Loop N1_A elements along row i0 of A and down column i1 of B
        for (size_t n=0; n<N1_A; n++) {

            //// Exercise, fix these two wrong lines of code ////
            
            //// Hint: Ask these questions
            //// What is the starting position for A?
            //// What is the stride along dimension 1 of A?
            //// What is the starting position for B?
            //// What is the stride along dimension 0 of B?
            
            size_t offset_A = 0;
            size_t offset_B = 0;
            
            //// End exercise ////

            // Add to the temporary sum
            temp+=A[offset_A]*B[offset_B];
        }

        // Place temp into the matrix at coordinates (i0, i1)
        C[i0*N1_C+i1]=temp;
    }
}    
```

The memory alloctions **A**, **B**, and **C** are passed into the function along with array sizes **NO_C**, **N1_C** and **N1_A** and coordinates **i0** and **i1**. As it stands the code runs but does not produce the right answer. Your task is replace the two lines of code for **offset_A** and **offset_B** in the kernel, so that the code accesses memory from the matrices **A_h** and **B_h** correctly, with no crashes and a very small or 0 error. We compile and run the code using this line below.

```bash
make clean; make
```

In [5]:
!make mat_mult.exe; ./mat_mult.exe

make: 'mat_mult.exe' is up to date.
The computed array (C_h) is
------
|  1.35e+00  1.35e+00  1.35e+00  1.35e+00  1.35e+00  1.35e+00 |
|  1.35e+00  1.35e+00  1.35e+00  1.35e+00  1.35e+00  1.35e+00 |
|  1.35e+00  1.35e+00  1.35e+00  1.35e+00  1.35e+00  1.35e+00 |
|  1.35e+00  1.35e+00  1.35e+00  1.35e+00  1.35e+00  1.35e+00 |
------
The solution array (C_answer_h) is 
------
|  8.33e-01  4.66e-01  8.74e-01  6.26e-01  4.29e-01  3.82e-01 |
|  2.38e-01  1.33e-01  2.72e-01  2.09e-01  1.45e-01  1.24e-01 |
|  7.25e-01  2.97e-01  7.72e-01  8.87e-01  4.26e-01  3.45e-01 |
|  4.68e-01  3.12e-01  7.38e-01  5.46e-01  4.66e-01  3.80e-01 |
------
The residual (C_answer_h-C_h) is
------
| -5.18e-01 -8.85e-01 -4.78e-01 -7.25e-01 -9.22e-01 -9.69e-01 |
| -1.11e+00 -1.22e+00 -1.08e+00 -1.14e+00 -1.21e+00 -1.23e+00 |
| -6.26e-01 -1.05e+00 -5.79e-01 -4.64e-01 -9.26e-01 -1.01e+00 |
| -8.83e-01 -1.04e+00 -6.14e-01 -8.05e-01 -8.85e-01 -9.71e-01 |
------
Maximum error (infinity norm) is: 1.2


The residual or difference between the computed array **C_h** and the array computed by a matrix multiplication library **C_answer_h** should be zero. When the two lines of code are fixed, the residual should be all zeros. It might be helpful to use **std::printf** or **std::cout** to print out variables for debugging.

### The answer

If you get stuck you can always peek at the answer in [mat_mult_answer.cpp](mat_mult_answer.cpp) and try to understand why the offsets are being calculated in they way they are. This is the output that the code should produce.

In [2]:
!make mat_mult_answer.exe; ./mat_mult_answer.exe

make: 'mat_mult_answer.exe' is up to date.
The computed array (C_h) is
------
|  8.33e-01  4.66e-01  8.74e-01  6.26e-01  4.29e-01  3.82e-01 |
|  2.38e-01  1.33e-01  2.72e-01  2.09e-01  1.45e-01  1.24e-01 |
|  7.25e-01  2.97e-01  7.72e-01  8.87e-01  4.26e-01  3.45e-01 |
|  4.68e-01  3.12e-01  7.38e-01  5.46e-01  4.66e-01  3.80e-01 |
------
The solution array (C_answer_h) is 
------
|  8.33e-01  4.66e-01  8.74e-01  6.26e-01  4.29e-01  3.82e-01 |
|  2.38e-01  1.33e-01  2.72e-01  2.09e-01  1.45e-01  1.24e-01 |
|  7.25e-01  2.97e-01  7.72e-01  8.87e-01  4.26e-01  3.45e-01 |
|  4.68e-01  3.12e-01  7.38e-01  5.46e-01  4.66e-01  3.80e-01 |
------
The residual (C_answer_h-C_h) is
------
|  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00 |
|  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00 |
|  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00 |
|  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00 |
------
Maximum error (infinity norm) is: 0
